In [1]:
import os
import pandas as pd
import pickle 
import numpy as np
from shogun import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import random

import time

In [3]:
import hypothesisTesting\mmd

SyntaxError: unexpected character after line continuation character (<ipython-input-3-3ac572843ce7>, line 1)